In [1]:
# import web driver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from Login_info import user as user1, password as pass1
import time, math
import pandas as pd

In [2]:
import linkedin_scraping as ls

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver',options=options)

In [4]:
time.sleep(3)
driver.get("https://www.linkedin.com")

time.sleep(1)

In [5]:
ls.Login(driver,user1,pass1)

In [ ]:
keyword = 'Applaudo Studios'  
posts = ls.Posts_pagination(driver, keyword)

In [ ]:
posts_df = pd.DataFrame(posts, columns=['post_url'])
posts_df.to_csv('data/posts_url.csv', index=False) 

In [6]:
posts_url_df = pd.read_csv('data/posts_url.csv')
posts_url_df.head()

,post_url
0,https://www.linkedin.com/feed/update/urn:li:ac...
1,https://www.linkedin.com/feed/update/urn:li:ac...
2,https://www.linkedin.com/feed/update/urn:li:ac...
3,https://www.linkedin.com/feed/update/urn:li:ac...
4,https://www.linkedin.com/feed/update/urn:li:ac...


In [7]:
def get_load_more_btn(wd):
    try:
        return wd.find_element_by_xpath("//*[text()='Load more comments']")
    except:
        return 'Element not found'


def get_load_previous_btn(wd):
    try:
        return wd.find_element_by_xpath("//*[text()='Load previous replies']")
    except:
        return 'Element not found'
    
def get_post_comments(wd):
    commentors_name = []
    commentors_href = []
    comments = []
    replies = []
    number_of_replies = []

    commentors_name_class_name = 'comments-post-meta__name-text.hoverable-link-text.mr1'
    commentors_href_class_name = 'ember-view.comments-post-meta__profile-link.t-16.t-black.t-bold.tap-target'
    comments_class_name = 'comments-comment-item__main-content.feed-shared-main-content--comment.t-14.t-black.t-normal'
    close_modal_btn_cn = 'artdeco-modal__dismiss.artdeco-button.artdeco-button--circle.artdeco-button--muted.artdeco-button--2.artdeco-button--tertiary.ember-view'
    
    try:
        wd.find_element_by_class_name(close_modal_btn_cn).click()
        time.sleep(1)
    except:
        pass

    # loading all comments
    load_more_btn = get_load_more_btn(wd)
    load_previous_btn = get_load_previous_btn(wd)

    comments_dataframe = pd.DataFrame(
        columns=['post_url', 'username', 'profile_url', 'comment', 'number_replies'])

    if len(wd.find_elements_by_class_name('comments-comments-list')) == 0:
        comments_dataframe = comments_dataframe.append({'post_url': wd.current_url,
                                                        'username': '', 'profile_url': '', 'comment': '', 'number_replies': ''},
                                                       ignore_index=True)
        return comments_dataframe

    while(load_more_btn != 'Element not found'):
        load_more_btn.click()
        time.sleep(3)
        load_more_btn = get_load_more_btn(wd)

    while(load_previous_btn != 'Element not found'):
        load_previous_btn.click()
        time.sleep(3)
        load_previous_btn = get_load_previous_btn(wd)

    time.sleep(1)

    amount_of_main_comments = wd.find_elements_by_class_name(
        "comments-comment-item.comments-comments-list__comment-item")

    for commentor in wd.find_elements_by_class_name(commentors_name_class_name):
        commentors_name.append(commentor.text)

    for commentor in wd.find_elements_by_class_name(commentors_href_class_name):
        commentors_href.append(commentor.get_attribute('href'))

    for comment in wd.find_elements_by_class_name(comments_class_name):
        comments.append(comment.text)

    for i in range(1, len(amount_of_main_comments)+1):
        try:
            replies_of_comment = wd.find_element_by_xpath(
                f"//article[{i}]/div[6]/div[3]/div/div[3]/span[2]")
            replies.append(int(replies_of_comment.text.split(' ')[0]))
        except:
            replies.append(0)

    for i in replies:
        word = 'Reply' if i == 1 else 'Replies'
        number_of_replies.append(f'{i} {word}')

        for k in range(i):
            number_of_replies.append('Is a Reply')

    comments_dataframe['post_url'] = [
        wd.current_url for i in range(len(commentors_name))]
    comments_dataframe['username'] = commentors_name
    comments_dataframe['profile_url'] = commentors_href
    comments_dataframe['comment'] = comments
    comments_dataframe['number_replies'] = number_of_replies

    return comments_dataframe


In [8]:
def get_post_reactions(wd):
    reactors_name = []
    reactors_href = []
    reactions_type = []

    reactions_window_btn = 'v-align-middle.social-details-social-counts__reactions-count'
    reactions_window_class_name = 'artdeco-modal__content.social-details-reactors-modal__content.ember-view'
    reactors_href_class_name = 'link-without-hover-state.ember-view'
    reactors_name_class_name = 'artdeco-entity-lockup__title.ember-view'
    reaction_class_name = 'reactions-icon.social-details-reactors-tab-body__icon.reactions-icon__consumption--small'
    modal_reactions_cn = 'social-details-reactors-tab-body'

    reactions_dataframe = pd.DataFrame(
        columns=['post_url', 'username', 'profile_url', 'reaction'])

    # opening reactions window
    try:
        reactions = wd.find_element_by_class_name(reactions_window_btn)
    except:
        reactions_dataframe = reactions_dataframe.append({'post_url' : wd.current_url,
                    'username' : '', 'profile_url': '', 'reaction':''} , 
                    ignore_index=True)
        return reactions_dataframe

    n_reactions = int(reactions.text)
    reactions.click()

    time.sleep(2)

    reactions_modal = wd.find_element_by_xpath(
        "//div[@class='social-details-reactors-tab-body']")

    # scroll down in reactions window
    panel = wd.find_element_by_class_name(reactions_window_class_name)
    for i in range(int(n_reactions/10)+1):
        wd.execute_script(
            'arguments[0].scrollTop = arguments[0].scrollHeight', panel)
        time.sleep(2)
        try:
            wd.find_element_by_xpath("//*[text()='Show more results']").click()
        except:
            continue

    for reactor in reactions_modal.find_elements_by_class_name(reactors_href_class_name):
        reactors_href.append(reactor.get_attribute('href'))

    for reactor in reactions_modal.find_elements_by_class_name(reactors_name_class_name):
        reactors_name.append(reactor.text)

    for reaction in reactions_modal.find_elements_by_class_name(reaction_class_name):
        reactions_type.append(reaction.get_attribute('alt'))

    reactions_dataframe['post_url'] = [
        wd.current_url for i in range(len(reactors_name))]
    reactions_dataframe['username'] = reactors_name
    reactions_dataframe['profile_url'] = reactors_href
    reactions_dataframe['reaction'] = reactions_type

    return reactions_dataframe

In [82]:
def get_media(wd, post_element):
    media_url = ''
    media_type = 'Unknown' # unknown o no media?
    images = post_element.find_elements_by_class_name('ivm-view-attr__img--centered.feed-shared-image__image.lazy-image.ember-view')
    if len(images) == 1:
        media_url = images[0].get_attribute('src')
        media_type = 'Image'
    elif len(images) > 1:
        media_url = []
        for img in images:
            media_url.append(img.get_attribute('src'))
        media_type = 'Images'
    else:
        wd.switch_to.frame(driver.find_element_by_tag_name("iframe"))
        carousel = wd.find_elements_by_class_name('carousel-lazy-element')
        if len(carousel) > 0:
            media_url = []
            for slide in carousel:
                media_url.append(slide.get_attribute('src'))
            media_type = 'Carousel'
            wd.switch_to.default_content()
        else:
            wd.switch_to.default_content()
            try:
                media_url = post_element.find_element_by_class_name('vjs-tech').get_attribute('src')
                media_type = 'Video'
            except:
                try:
                    media_url = post_element.find_element_by_class_name('tap-target.feed-shared-mini-update-v2__link-to-details-page.text-body-medium.ember-view').get_attribute('href')
                    media_type = 'Reshared post'
                except:
                    pass
                
    return media_url, media_type
        

def get_post(wd):
    post_df = pd.DataFrame(
        columns=['post_url', 'author', 'author_url', 'author_type', 'description', 'media_type', 'media_url'])
    
    author_element_cn = 'app-aware-link.feed-shared-actor__container-link.relative.display-flex.flex-grow-1'
    author_name_cn = 'feed-shared-actor__name.t-14.t-bold.hoverable-link-text.t-black'
    post_description_cn = 'feed-shared-inline-show-more-text.feed-shared-update-v2__description.feed-shared-inline-show-more-text--minimal-padding.feed-shared-inline-show-more-text--expanded'
    
    post_element = wd.find_element_by_class_name('core-rail.update-outlet')
    author_url = post_element.find_element_by_class_name(author_element_cn).get_attribute('href').split('?')[0]
    author_name = post_element.find_element_by_class_name(author_name_cn).text    
    author_type = 'Company' if author_url.find('company') != -1 else 'User'
    
    try:
        description = post_element.find_element_by_class_name(post_description_cn).text
    except:
        description = 'No description'
    
    try:
        source = post_element.get_attribute('src')
    except:
        source = 'No source'
    
    media_url, media_type = get_media(wd, post_element)
    
    post_df = post_df.append({'post_url' : wd.current_url, 'author' : author_name, 'author_url': author_url, 
                              'author_type': author_type, 'description': description, 'media_type': media_type,
                              'media_url': media_url } , 
                    ignore_index=True)
    print('media_url', media_url)
    print('media_type', media_type)

    return post_df

In [83]:
driver.switch_to.default_content()
post = get_post(driver)
post

media_url 
media_type Unknown


,post_url,author,author_url,author_type,description,media_type,media_url
0,https://www.linkedin.com/feed/update/urn:li:ac...,César Retana,https://www.linkedin.com/in/cretana,User,Today I got my first Performance Review delive...,Unknown,


In [77]:
post_description_cn = 'feed-shared-inline-show-more-text.feed-shared-update-v2__description.feed-shared-inline-show-more-text--minimal-padding.feed-shared-inline-show-more-text--expanded'

# post_element = driver.find_element_by_class_name('core-rail.update-outlet')
# cn = 'vjs-tech'
# post_element.find_elements_by_class_name(cn)[0].get_attribute('src')
driver.switch_to.default_content()

driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
src = driver.find_elements_by_class_name('carousel-lazy-element')[2].get_attribute('src')
# innerHTML = src.get_attribute('src')
print(src)

https://media-exp3.licdn.com/dms/image/C4D1FAQEfNOb1DM4-Gg/feedshare-document-images_800/3/1625675311741?e=1625878800&v=beta&t=agRSgBYetl_fDOIZjxSRm9soUthFPGEGm3mEHSa9rpU


In [76]:
driver.find_element_by_class_name('feed-shared-inline-show-more-text.feed-shared-update-v2__description.feed-shared-inline-show-more-text--minimal-padding.feed-shared-inline-show-more-text--expanded').text

'Check these open positions at Applaudo Studios\nApply at https://lnkd.in/dcNQa2x and submit your application'

In [15]:
posts_reactions = pd.DataFrame()
posts_comments = pd.DataFrame()

for i, row in posts_url_df.iterrows():
    if i > 4:
        break
    driver.get(row['post_url'])
    time.sleep(2)
#     post_r = ls.get_post_reactions(driver)
#     posts_reactions = pd.concat([posts_reactions,post_r]) 
#     post_c = ls.get_post_comments(driver)
#     posts_comments = pd.concat([posts_comments,post_c])
    print(i)
    

0
1
2
3
4


In [16]:
posts_reactions.tail()

,post_url,username,profile_url,reaction
114,https://www.linkedin.com/feed/update/urn:li:ac...,Daniela Bonet\nView Daniela Bonet’s profile,https://www.linkedin.com/in/ACoAACIiPDMBKJukwg...,EMPATHY
115,https://www.linkedin.com/feed/update/urn:li:ac...,"Bryan Velásquez, SMPC®, LSSGB\nView Bryan Velá...",https://www.linkedin.com/in/ACoAACQsGioBihF1BY...,LIKE
116,https://www.linkedin.com/feed/update/urn:li:ac...,Belly Oweer Espinal Larios\nView Belly Oweer E...,https://www.linkedin.com/in/ACoAACFQb98BoKiT7A...,PRAISE
0,https://www.linkedin.com/feed/update/urn:li:ac...,,,
0,https://www.linkedin.com/feed/update/urn:li:ac...,,,


In [17]:
posts_comments.tail()

,post_url,username,profile_url,comment,number_replies
8,https://www.linkedin.com/feed/update/urn:li:ac...,Gabriela Monroy,https://www.linkedin.com/in/gabriela-monroy-9b...,Great Job Erick! 🚀,1 Reply
9,https://www.linkedin.com/feed/update/urn:li:ac...,Erick Henríquez,https://www.linkedin.com/in/erick-henr%C3%ADqu...,Gracias Gaby!!,Is a Reply
10,https://www.linkedin.com/feed/update/urn:li:ac...,Aracely Ocón,https://www.linkedin.com/in/aracely-oc%C3%B3n-...,🤩👏🏻 Erick Henríquez,0 Replies
0,https://www.linkedin.com/feed/update/urn:li:ac...,,,,
0,https://www.linkedin.com/feed/update/urn:li:ac...,,,,


In [ ]:
reactions_window_btn = 'v-align-middle.social-details-social-counts__reactions-count'
driver.find_element_by_class_name(reactions_window_btn)

In [ ]:
# for commentor in wd.find_elements_by_class_name(commentors_name_class_name):
    #     commentors_name.append(commentor.text)
    
    
    183

In [ ]:
posts_comments.head()

In [ ]:


element2 = driver.find_element_by_xpath("//div[@class='social-details-reactors-tab-body']")
elements3 = element2.find_elements_by_xpath(".//div[@class='artdeco-entity-lockup__title ember-view']")

for element in elements3:
    print(element.text)
    

In [ ]:
# 'https://www.linkedin.com/posts/anaromerolinkepower_equipopower-familiaempresarial-activity-6817819334484774912-B4R2/'
# https://www.linkedin.com/posts/jorgebranger_nunca-dejes-pasar-una-oportunidad-por-estar-activity-6815238905818832896-ix4P/

post_comments = get_post_comments(driver)
post_comments.head()

In [ ]:
post_reactions_and_comments = pd.concat([post_reactions,post_comments])
post_reactions_and_comments.to_csv('C:/Users/VanessaNunez/Downloads')

In [ ]:
# fixed full
# feed-shared-update-v2__description-wrapper

In [ ]:
for k in reactors.keys():
    print(k)

In [ ]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [ ]:
driver.find_elements_by_class_name('social-details-reactors-tab-body')

In [ ]:
#driver.close()

In [ ]:
driver.window_handles

In [ ]:
driver.current_window_handle

In [ ]:
def Posts(wd):
    time.sleep(2)
    # Click in Posts
    posts_url = wd.current_url + "posts/"
    wd.get(posts_url)
    time.sleep(2)
    
    # Scroll till end of page
    inicial_yoffset = 0
    scrolled_yoffset = driver.execute_script("return window.pageYOffset;") + 1 # Inicial return is 0

    while scrolled_yoffset > inicial_yoffset:
        inicial_yoffset = driver.execute_script("return window.pageYOffset;")
        driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(1)
        scrolled_yoffset = driver.execute_script("return window.pageYOffset;")
        
    # String to look
    data_urn = 'urn:li:activity:'
    posts = {}
            
    # Get posts url
    for i in wd.find_elements_by_class_name("occludable-update.ember-view"):
        posts[i.get_attribute('href')] = 1

    # Click next page
    if j + 1 < n_pages:
        wd.find_elements_by_class_name('artdeco-pagination__button.artdeco-pagination__button--next.artdeco-button.artdeco-button--muted.artdeco-button--icon-right.artdeco-button--1.artdeco-button--tertiary.ember-view')[0].click()

    return posts
        